The main program that uses the database and defined python scripts to run the analysis

In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [ ]:
scores = []
scores = list(map(runSimulation, range(80), repeat(seasonsData), repeat(qualiResultsData), repeat(driversData), 
                    repeat(constructorsData), repeat(enginesData)))
print(np.median(scores))

So, now when the data has been analysed, it's time to use it to predict the next qualification's results! In this simplified version, it will be the Australian GP 2018.

In [3]:
(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [4]:
newDrivers = json.load(open('newDrivers.json'))["drivers"]
#print(newDrivers)

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [5]:
circuit = 4

driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 13.9 %
	2: 27.3 %
	3: 21.9 %
	4: 17.4 %
	5: 11.5 %
	6: 6.0 %
	7: 0.1 %
	11: 0.1 %
	18: 0.3 %
	19: 0.4 %
	20: 1.1 %
Fernando Alonso (McLaren):
	6: 0.1 %
	7: 1.6 %
	8: 7.0 %
	9: 16.1 %
	10: 19.0 %
	11: 17.4 %
	12: 15.1 %
	13: 11.2 %
	14: 7.0 %
	15: 3.0 %
	16: 0.4 %
	17: 0.2 %
	18: 0.1 %
	19: 0.5 %
	20: 1.3 %
Kimi Räikkönen (Ferrari):
	1: 28.4 %
	2: 29.1 %
	3: 18.0 %
	4: 11.0 %
	5: 7.2 %
	6: 4.2 %
	7: 0.2 %
	18: 0.5 %
	19: 0.5 %
	20: 0.9 %
Sebastian Vettel (Ferrari):
	1: 52.4 %
	2: 20.9 %
	3: 11.3 %
	4: 6.1 %
	5: 4.3 %
	6: 2.7 %
	18: 0.2 %
	19: 1.2 %
	20: 0.9 %
Romain Grosjean (Haas F1 Team):
	6: 0.1 %
	7: 0.6 %
	8: 4.9 %
	9: 10.6 %
	10: 13.1 %
	11: 20.1 %
	12: 18.6 %
	13: 14.5 %
	14: 10.4 %
	15: 3.2 %
	16: 1.2 %
	17: 0.3 %
	18: 0.2 %
	19: 0.4 %
	20: 1.8 %
Nico Hülkenberg (Renault):
	4: 0.2 %
	5: 0.5 %
	6: 10.1 %
	7: 48.5 %
	8: 22.5 %
	9: 6.2 %
	10: 3.5 %
	11: 1.8 %
	12: 2.3 %
	13: 1.2 %
	14: 0.6 %
	15: 0.2 %
	17: 0.1 %
	18: 0.1 %
	19: 0.9 %
	20: 1.3 %
Sergi